In [3]:
import sqlalchemy 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, text
import os
import pandas as pd
import csv
import json

In [4]:
csv_file = os.path.join("state_unemployment.csv")
state_unemployment_df = pd.read_csv(csv_file)
state_unemployment_df.head()

,State,Oct-10,Nov-10,Dec-10,Jan-11,Feb-11,Mar-11,Apr-11,May-11,Jun-11,...,Jan-20,Feb-20,Mar-20,Apr-20,May-20,Jun-20,Jul-20,Aug-20,Sep-20,Oct-20
0,Alabama,10.1,10.2,10.3,10.2,10.2,10.1,10.0,10.0,10.0,...,2.7,2.7,3,13.8,9.6,7.6,7.9,5.6,6.7,5.8
1,Alaska,7.8,7.8,7.8,7.8,7.7,7.7,7.6,7.6,7.5,...,6.0,5.8,5.2,13.5,12.7,12.4,11.6,7.4,7.2,5.9
2,Arizona,10.1,10.0,9.9,9.8,9.7,9.7,9.7,9.7,9.7,...,4.5,4.5,6.1,13.4,9.0,10.0,10.7,5.9,6.5,8.0
3,Arkansas,8.3,8.3,8.4,8.5,8.5,8.5,8.5,8.5,8.4,...,3.5,3.5,5,10.8,9.6,8.1,7.1,7.4,7.2,6.2
4,California,12.3,12.3,12.2,12.1,12.0,11.9,11.8,11.8,11.8,...,3.9,3.9,5.5,16.4,16.4,14.9,13.5,11.2,11.1,9.3


In [5]:
state_unemployment_df.dtypes

State      object
Oct-10    float64
Nov-10    float64
Dec-10    float64
Jan-11    float64
           ...   
Jun-20    float64
Jul-20    float64
Aug-20    float64
Sep-20    float64
Oct-20    float64
Length: 122, dtype: object

In [6]:
state_unemployment_df.to_dict(orient="records")

[{'State': 'Alabama',
  'Oct-10': 10.1,
  'Nov-10': 10.2,
  'Dec-10': 10.3,
  'Jan-11': 10.2,
  'Feb-11': 10.2,
  'Mar-11': 10.1,
  'Apr-11': 10.0,
  'May-11': 10.0,
  'Jun-11': 10.0,
  'Jul-11': 9.9,
  'Aug-11': 9.7,
  'Sep-11': 9.4,
  'Oct-11': 9.0,
  'Nov-11': 8.6,
  'Dec-11': 8.3,
  'Jan-12': 8.1,
  'Feb-12': 7.9,
  'Mar-12': 8.0,
  'Apr-12': 8.1,
  'May-12': 8.2,
  'Jun-12': 8.3,
  'Jul-12': 8.2,
  'Aug-12': 8.1,
  'Sep-12': 7.9,
  'Oct-12': 7.8,
  'Nov-12': 7.6,
  'Dec-12': 7.6,
  'Jan-13': 7.5,
  'Feb-13': 7.4,
  'Mar-13': 7.3,
  'Apr-13': 7.2,
  'May-13': 7.1,
  'Jun-13': 7.0,
  'Jul-13': 7.1,
  'Aug-13': 7.1,
  'Sep-13': 7.2,
  'Oct-13': 7.2,
  'Nov-13': 7.2,
  'Dec-13': 7.3,
  'Jan-14': 7.2,
  'Feb-14': 7.2,
  'Mar-14': 7.2,
  'Apr-14': 7.1,
  'May-14': 7.0,
  'Jun-14': 6.9,
  'Jul-14': 6.7,
  'Aug-14': 6.6,
  'Sep-14': 6.5,
  'Oct-14': 6.4,
  'Nov-14': 6.2,
  'Dec-14': 6.1,
  'Jan-15': 6.0,
  'Feb-15': 6.0,
  'Mar-15': 6.1,
  'Apr-15': 6.1,
  'May-15': 6.2,
  'Jun-15': 6.2,


In [5]:
pg_user = 'postgres'
pg_password = 'Watermelontessa7'
db_name = 'unemployment_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

state_unemployment_df.to_sql(name='states', con=engine, if_exists='append', index=False)

In [6]:
# Read in unemployment file and insert into table
csvFile = open('state_unemployment.csv', 'r')
reader = csv.DictReader(csvFile)
for row in reader:
  state = row['State']
  del row['State']
  month_data = json.dumps(row)

  # insert into the database
  query = text('INSERT INTO states_unemployment (state, data) VALUES (:state, :data)')
  engine.execute(query, state=state, data=month_data)

In [7]:
# Read in the geojson file and insert it into the table
json_file = open('statesdata.geojson', 'r')
data = json.load(json_file)

for row in data['features']:
  id = int(row['id'])
  state = row['properties']['name']
  density = row['properties']['density']
  coordinates = json.dumps(row['geometry']['coordinates'])
  
  query = text('INSERT INTO geojson (id, state, density, coordinates) VALUES (:id, :state, :density, :coordinates)')
  engine.execute(query, id=id, state=state, density=density, coordinates=coordinates)

In [8]:
# select the state data
with engine.connect() as connection:
  result = connection.execute('SELECT geojson.state, geojson.density, geojson.coordinates, states_unemployment.data FROM geojson INNER JOIN states_unemployment on geojson.state = states_unemployment.state')
  # what to return from api
  result = [dict(row) for row in result]


  print(type(result[0]))

<class 'dict'>


In [9]:
engine.table_names()

['states_unemployment', 'geojson', 'states']

In [2]:
pd.read_sql_query('select * from states_unemployment', con=engine).head()

NameError: name 'pd' is not defined